### VGGNet-16网络搭建

In [2]:
from torch import nn
import torch

In [1]:
class VGG(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG, self).__init__()
        layers = []
        in_dim = 3
        out_dim = 64
        
        # 循环构造卷积层 共有13个卷积层
        for i in range(13):
            layers += [
                nn.Conv2d(in_dim, out_dim, 3, 1, 1),
                nn.ReLU(inplace=True)
            ]
            in_dim = out_dim
            # 在第2、4、7、10、13增加池化层
            if i == 1 or i == 3 or i == 6 or i == 9 or i == 12:
                layers += [nn.MaxPool2d(2, 2)]
                # 第10个卷积层保持和前面的通道数一致，都为512
                if i != 9:
                    out_dim *= 2
        self.features = nn.Sequential(*layers)
        # VGGNet的3个全连接层，中间有Relu和Dropout层
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        # 将特征图的维度[1,512,7,7]转换为[1,512*7*7]
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [3]:
'''
实例化vgg
'''

vgg = VGG(10)
x = torch.randn(1, 3, 224, 224)
y = vgg(x)
y.shape

torch.Size([1, 10])

In [4]:
'''
单独使用vgg部件
'''

features = vgg.features(x)
features.shape

torch.Size([1, 512, 7, 7])

In [5]:
'''
打印特征层和分类层
'''

vgg.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(

In [6]:
vgg.features[0]

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [7]:
vgg.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=10, bias=True)
)